Display the vocabulary and frequency of each word in main page and all other documents linked from main page. Also for each word compute in how many documents the word occurs.

In [1]:
def get_text_only(url,doc_no,my_df):
    res=requests.get(url)
    soup=BeautifulSoup(res.text)
    text=soup.get_text()
    words=re.findall(r'[^\W\d]+',text)
    my_df['words']=words
    my_df['words']=my_df['words'].apply(lambda x: str.capitalize(x))
    my_df['count']=my_df['words'].apply(lambda x : my_df[my_df['words']==x].count().to_list()[0])
    my_df['document']=[doc_no]*len(words)
    my_df.drop_duplicates(inplace=True)
    my_df.sort_values(by='words',ascending=True,inplace=True)
    return my_df

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

url='https://www.memphis.edu/datascience/'

res=requests.get(url)

soup=BeautifulSoup(res.text)

links=soup.find_all('a',href=True)

all_urls=['https://www.memphis.edu/datascience/']

for link in links:
    if(link.get('href').strip().startswith('https')):
        all_urls.append(link.get('href').strip())
        
all_urls=list(set(all_urls))

all_words=pd.DataFrame(columns=['words','count','document'])

for index in range(len(all_urls)):
    #basic_df=pd.DataFrame(columns=['words','count','document'])
    all_words=pd.concat([all_words,get_text_only(all_urls[index],index+1,pd.DataFrame())],axis=0,ignore_index=True)
    

In [4]:
def func(df):
    count=','.join(str(x) for x in df['count'].to_list())
    document=','.join(str(x) for x in df['document'].to_list())
    return count,document,len(df['count'].to_list())

index_to_drop=all_words[all_words['words'].apply(lambda x : len(x)==1)].index
all_words.drop(index_to_drop,axis=0,inplace=True)
all_words['No. of webpages']=[0]*all_words.shape[0]
all_words=all_words.groupby('words',as_index=False)['count','document','No. of webpages'].apply(func)
all_words.drop_duplicates(inplace=True)
all_words.to_csv('multiple_webpages.csv')
all_words


,words,count,document,No. of webpages
0,Aa,"3,1,3","66,79,93",3
1,Aac,1,85,1
2,Aba,"1,1","25,89",2
3,Ability,"1,1,1,1,1,1,1,1,2","7,11,25,28,69,78,81,89,115",9
4,Able,"1,1,1,1,1,1","2,20,50,52,79,82",6
...,...,...,...,...
4501,Yuan,1,23,1
4502,Zhang,3,23,1
4503,Zone,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,...","2,3,5,6,7,8,9,10,12,13,15,17,18,19,20,21,22,23...",90
4504,Zoom,"1,1","26,95",2
